In [1]:
import keras
import numpy as np
import keras.preprocessing.text
from sklearn.preprocessing import OneHotEncoder
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Activation, Lambda, Dropout
from keras.layers.recurrent import SimpleRNN

Using TensorFlow backend.


In [2]:
with open('data/input.txt','U') as f:
    content = f.read()

In [4]:
np.random.seed = 0
def createDict(content, start=0):
    char_to_num = {}
    num_to_char = {}
    content = set(list(content))
    for c in content:
        char_to_num[c] = start
        num_to_char[start] = c
        start += 1
    return (char_to_num, num_to_char)

def charToNum(content, char_to_num):
    content = list(content)
    res = []
    for c in content:
        res.append(char_to_num[c])
    return res

def numToChar(content, num_to_char):
    res = []
    for i in content:
        res.append(num_to_char[i])
    return res

def cateToNum(content):
    res = []
    for line in content:
        idx = 0
        for i in line:
            if i == 0:
                idx +=1
            else:
                res.append(idx)
                break
    return res

def createData(data, size=50000):
    d = []
    for idx in range(size):
        r = np.random.randint(0, len(data)-25)
        d.append(data[r:r+25,:])
    d = np.array(d)
    d_input = d[:,:-1,]
    d_output = d[:,1:,]
    return (d_input, d_output)

def splitData(data, rate=0.8):
    idx = int(len(data)*0.8)
    train_data = data[:idx]
    test_data = data[idx:]
    return (train_data, test_data)

In [5]:
char_to_num, num_to_char = createDict(content)
oneHotEncoding = charToNum(content, char_to_num)

In [ ]:
data = np_utils.to_categorical(oneHotEncoding, nb_classes=100)
train_data, test_data = splitData(data)

In [ ]:
train_input,train_output = createData(train_data, 50000)
test_input, test_output = createData(test_data, int(50000*0.25))

In [16]:
model = keras.models.load_model('data/4d_0_1.h5')

In [17]:
def sample(probDist, temperature=1.0):
    # helper function to sample an index from a probability array
    a = [x/sum(probDist) for x in probDist]
    idx = 93
    while idx > 92:
        idx = np.argmax(np.random.multinomial(1, a, 1))
    return idx

lenGenerate = 20000
generated = ''
sentence = content[:24]
for i in range(lenGenerate):
    x = np.zeros((1, 24, 100))
    for t, char in enumerate(sentence):
        x[0, t, char_to_num[char]] = 1.

    tmp = model.predict(x, verbose=0)
    preds = tmp[0][-1]
    next_index = sample(preds)
    next_char = num_to_char[next_index]

    generated += next_char
    sentence = sentence[1:] + next_char
with open('generate_0_1.txt', "w") as text_file:
    text_file.write("{0}".format(generated))